# <a id="top"></a>Autoencoding Edward Hopper:<br>Using deep learning to recommend art

[Larry Finer](mailto:lfiner@gmail.com)  
March 2019

The goal of this project was to build a model that would take an image of an artwork and compare it visually to a corpus of more than 100,000 artworks from museums and other sources in order to find works that are similar visually. The main steps in the project were:

1. <b>Download artwork images and metadata from multiple sites</b> (this file)  
2. Combine metadata into a single pandas dataframe  
3. Develop a convolutional neural network autoencoder model that adequately reproduces the images
4. Extract the narrowest encoded layer and use it to encode the entire corpus as well as a test image; then compare the test image to the entire corpus using a cosine distance measure to find the nearest images

<hr>

## 1b. Download Guggenheim images and metadata

This file downloads the Guggenheim museum's entire online collection of approximately 1,900 images, and also downloads relevant metadata for each artwork, including artist, title, date, and medium.

### Sections
1b1. [Imports and setup](#1b1)  
1b2. [Download images and metadata](#1b2)

### <a id="1b1"></a>1b1. Imports and setup

In [3]:
import os
import re
import pandas as pd
import numpy as np
import random
import time
import pickle
import urllib.request
import requests
from fake_useragent import UserAgent
from itertools import islice
from lxml import html
from bs4 import BeautifulSoup

In [5]:
# Create a user agent for web scraping
ua = UserAgent()
user_agent = {'User-agent': ua.random}

### <a id="1b2"></a>1b2. Download images and metadata

In [86]:
# Initialize counters for the main web scraping loop
idcounter = 0
artworkcounter = 0
imagecounter = 0

# Initialize an empty list that will contain the artwork metadata
gugg = []

On the Guggenheim website, artworks are presented on pages that have URLs of the form https://www.guggenheim.org/artwork/artwork-ID-number. Exploration revealed that all artworks have an ID number in the range 1 to 60,000. Rather than searching to find these ID numbers, I simply tried each of the 60,000 URLs.

In [9]:
# Create a list of 60,000 ids in random order
ids = list(range(60000))
random.Random(42).shuffle(ids)

# Take a look at a few
ids[:10]

[15586, 33591, 37199, 23870, 29869, 8176, 17876, 32671, 13543, 5084]

In [3]:
# IF program was interrupted, run this cell to reload the saved counters and list of artworks
idcounter = pickle.load(open('./data/gugg/ID counter.pickle', 'rb'))
artworkcounter = pickle.load(open('./data/gugg/Artwork counter.pickle', 'rb'))
imagecounter = pickle.load(open('./data/gugg/Image counter.pickle', 'rb'))
gugg = pickle.load(open('./data/gugg/Guggenheim.pickle', 'rb'))

Here is the main web-scraping loop.

In [ ]:
# Short function to save counters and artwork list periodically.
def save_counters_and_artwork_list():
    pickle.dump(idcounter, open('./data/gugg/ID counter.pickle', 'wb'))
    pickle.dump(artworkcounter, open('./data/gugg/Artwork counter.pickle', 'wb'))
    pickle.dump(imagecounter, open('./data/gugg/Image counter.pickle', 'wb'))        
    pickle.dump(gugg, open('./data/gugg/Guggenheim.pickle', 'wb'))
    print()
    print('IDs:', idcounter)
    print('Artworks:', artworkcounter)
    print('Images:', imagecounter)
    print()

In [10]:
print('Starting count for this run:', idcounter)
print()
# Try all id numbers from 1 (or most recently saved counter) to 60,000
for id in ids[idcounter:]:
    print('Attempt:', str(idcounter), 'Artwork ID:', str(id))
    url = 'https://www.guggenheim.org/artwork/' + str(id)

    timeDelay = random.randrange(20, 30)/1000
    time.sleep(timeDelay)
    
    # Download the page contents using Beautiful Soup
    soup = BeautifulSoup(requests.get(url, headers = user_agent).text, "lxml")

    # If there is a link with rel='canonical', this is an artwork page.
    try:
        pagestring = soup.find('link', rel='canonical')['href']
        # Try to get the ID number.
        pageid = int(pagestring[pagestring.rfind('/')+1:])
    # If the above failed, it is not an artwork page.
    except:
        pagestring = None
    # If not an artwork page or no artwork ID:
    if pagestring == None or pageid != id:
        print('== 404 or wrong page ==')
        # Save the counters and artwork list every 20 tries.
        if idcounter % 20 == 0:
            save_counters_and_artwork_list()
        idcounter += 1
        continue
    
    # Create an empty dictionary for this artwork.
    newdict = {}

    # Save the artwork ID.
    newdict['artwork_uid'] = id
    
    # Extract artwork metadata: title, date and artist.
    newdict['title'] = soup.find('meta', property='og:title')['content']
    newdict['date'] = soup.find('meta', property='article:published_time')['content']

    datadump = soup.find('script', text=re.compile('"artist":')).text
    chop = datadump[datadump.find('"artist":[{"id":')+16:datadump.find(',"url"')]
    artist = chop[chop.find('"name":"')+8:-1]
    print(artist)
    newdict['artist'] = artist
    
    # Get the URL for the image.
    image_url = soup.find('meta', property='og:image:secure_url')['content']
    
    # Save the image itself into a local directory.
    img_data = requests.get(image_url, headers = user_agent).content
    file = './data/gugg/images/' + str(id) + '.jpg'
    # print(file)
    with open(file, 'wb') as handler:
        handler.write(img_data)
    imagecounter += 1
    
    # Save the artwork URL.
    newdict['image_url'] = image_url

    # Add the filled-in dictionary to the list of artworks.
    gugg.append(newdict)    
            
    # Increment counters.
    idcounter += 1
    artworkcounter += 1
    
    # Save counters and artwork list every 20 tries.
    if idcounter % 20 == 0:
        save_counters_and_artwork_list()

Starting count for this run: 10240

Attempt: 10240 Artwork ID: 15862
== 404 or wrong page ==

IDs: 10240
Artworks: 484
Images: 484

Attempt: 10241 Artwork ID: 5114
== 404 or wrong page ==
Attempt: 10242 Artwork ID: 39095
== 404 or wrong page ==
Attempt: 10243 Artwork ID: 7678
== 404 or wrong page ==
Attempt: 10244 Artwork ID: 37283
== 404 or wrong page ==
Attempt: 10245 Artwork ID: 31455
== 404 or wrong page ==
Attempt: 10246 Artwork ID: 4593
== 404 or wrong page ==
Attempt: 10247 Artwork ID: 8529
== 404 or wrong page ==
Attempt: 10248 Artwork ID: 5441
== 404 or wrong page ==
Attempt: 10249 Artwork ID: 23742
== 404 or wrong page ==
Attempt: 10250 Artwork ID: 18512
== 404 or wrong page ==
Attempt: 10251 Artwork ID: 13870
== 404 or wrong page ==
Attempt: 10252 Artwork ID: 38367
== 404 or wrong page ==
Attempt: 10253 Artwork ID: 39318
== 404 or wrong page ==
Attempt: 10254 Artwork ID: 29563
== 404 or wrong page ==
Attempt: 10255 Artwork ID: 29752
== 404 or wrong page ==
Attempt: 10256 Art

== 404 or wrong page ==
Attempt: 10382 Artwork ID: 6331
== 404 or wrong page ==
Attempt: 10383 Artwork ID: 22909
== 404 or wrong page ==
Attempt: 10384 Artwork ID: 14580
== 404 or wrong page ==
Attempt: 10385 Artwork ID: 33368
== 404 or wrong page ==
Attempt: 10386 Artwork ID: 28106
== 404 or wrong page ==
Attempt: 10387 Artwork ID: 33781
== 404 or wrong page ==
Attempt: 10388 Artwork ID: 20866
== 404 or wrong page ==
Attempt: 10389 Artwork ID: 18581
== 404 or wrong page ==
Attempt: 10390 Artwork ID: 2820
Andr\u00e9 Masson
Attempt: 10391 Artwork ID: 29300
== 404 or wrong page ==
Attempt: 10392 Artwork ID: 33744
== 404 or wrong page ==
Attempt: 10393 Artwork ID: 30131
== 404 or wrong page ==
Attempt: 10394 Artwork ID: 19807
== 404 or wrong page ==
Attempt: 10395 Artwork ID: 16908
== 404 or wrong page ==
Attempt: 10396 Artwork ID: 38500
== 404 or wrong page ==
Attempt: 10397 Artwork ID: 6530
== 404 or wrong page ==
Attempt: 10398 Artwork ID: 10544
== 404 or wrong page ==
Attempt: 10399 A

== 404 or wrong page ==
Attempt: 10524 Artwork ID: 13056
== 404 or wrong page ==
Attempt: 10525 Artwork ID: 37897
== 404 or wrong page ==
Attempt: 10526 Artwork ID: 14675
== 404 or wrong page ==
Attempt: 10527 Artwork ID: 23890
== 404 or wrong page ==
Attempt: 10528 Artwork ID: 32569
== 404 or wrong page ==
Attempt: 10529 Artwork ID: 9654
== 404 or wrong page ==
Attempt: 10530 Artwork ID: 21250
== 404 or wrong page ==
Attempt: 10531 Artwork ID: 32948
== 404 or wrong page ==
Attempt: 10532 Artwork ID: 25334
== 404 or wrong page ==
Attempt: 10533 Artwork ID: 32584
== 404 or wrong page ==
Attempt: 10534 Artwork ID: 29487
== 404 or wrong page ==
Attempt: 10535 Artwork ID: 21794
Richard Serra
Attempt: 10536 Artwork ID: 15993
== 404 or wrong page ==
Attempt: 10537 Artwork ID: 4551
== 404 or wrong page ==
Attempt: 10538 Artwork ID: 8874
== 404 or wrong page ==
Attempt: 10539 Artwork ID: 14141
== 404 or wrong page ==
Attempt: 10540 Artwork ID: 24703
== 404 or wrong page ==

IDs: 10540
Artworks

== 404 or wrong page ==
Attempt: 10665 Artwork ID: 38341
== 404 or wrong page ==
Attempt: 10666 Artwork ID: 36093
== 404 or wrong page ==
Attempt: 10667 Artwork ID: 30904
== 404 or wrong page ==
Attempt: 10668 Artwork ID: 38951
== 404 or wrong page ==
Attempt: 10669 Artwork ID: 4806
== 404 or wrong page ==
Attempt: 10670 Artwork ID: 4569
== 404 or wrong page ==
Attempt: 10671 Artwork ID: 8458
== 404 or wrong page ==
Attempt: 10672 Artwork ID: 11580
== 404 or wrong page ==
Attempt: 10673 Artwork ID: 32205
== 404 or wrong page ==
Attempt: 10674 Artwork ID: 3094
== 404 or wrong page ==
Attempt: 10675 Artwork ID: 26345
== 404 or wrong page ==
Attempt: 10676 Artwork ID: 38447
== 404 or wrong page ==
Attempt: 10677 Artwork ID: 38867
== 404 or wrong page ==
Attempt: 10678 Artwork ID: 29467
== 404 or wrong page ==
Attempt: 10679 Artwork ID: 4118
== 404 or wrong page ==
Attempt: 10680 Artwork ID: 18142
== 404 or wrong page ==

IDs: 10680
Artworks: 499
Images: 499

Attempt: 10681 Artwork ID: 278

== 404 or wrong page ==
Attempt: 10807 Artwork ID: 16062
== 404 or wrong page ==
Attempt: 10808 Artwork ID: 11628
== 404 or wrong page ==
Attempt: 10809 Artwork ID: 12475
== 404 or wrong page ==
Attempt: 10810 Artwork ID: 884
== 404 or wrong page ==
Attempt: 10811 Artwork ID: 6934
== 404 or wrong page ==
Attempt: 10812 Artwork ID: 15258
== 404 or wrong page ==
Attempt: 10813 Artwork ID: 18088
== 404 or wrong page ==
Attempt: 10814 Artwork ID: 37942
== 404 or wrong page ==
Attempt: 10815 Artwork ID: 6933
== 404 or wrong page ==
Attempt: 10816 Artwork ID: 35967
== 404 or wrong page ==
Attempt: 10817 Artwork ID: 26655
== 404 or wrong page ==
Attempt: 10818 Artwork ID: 27278
== 404 or wrong page ==
Attempt: 10819 Artwork ID: 16526
== 404 or wrong page ==
Attempt: 10820 Artwork ID: 27286
== 404 or wrong page ==

IDs: 10820
Artworks: 505
Images: 505

Attempt: 10821 Artwork ID: 28543
== 404 or wrong page ==
Attempt: 10822 Artwork ID: 38631
Sable Elyse Smith
Attempt: 10823 Artwork ID: 6135
== 

...

== 404 or wrong page ==
Attempt: 39424 Artwork ID: 1280
== 404 or wrong page ==
Attempt: 39425 Artwork ID: 38278
== 404 or wrong page ==
Attempt: 39426 Artwork ID: 25524
== 404 or wrong page ==
Attempt: 39427 Artwork ID: 27534
== 404 or wrong page ==
Attempt: 39428 Artwork ID: 13555
== 404 or wrong page ==
Attempt: 39429 Artwork ID: 4080
== 404 or wrong page ==
Attempt: 39430 Artwork ID: 27248
== 404 or wrong page ==
Attempt: 39431 Artwork ID: 30534
== 404 or wrong page ==
Attempt: 39432 Artwork ID: 35560
== 404 or wrong page ==
Attempt: 39433 Artwork ID: 21684
== 404 or wrong page ==
Attempt: 39434 Artwork ID: 26712
== 404 or wrong page ==
Attempt: 39435 Artwork ID: 27188
== 404 or wrong page ==
Attempt: 39436 Artwork ID: 4209
== 404 or wrong page ==
Attempt: 39437 Artwork ID: 14021
Willem de Kooning
Attempt: 39438 Artwork ID: 10014
== 404 or wrong page ==
Attempt: 39439 Artwork ID: 25102
== 404 or wrong page ==
Attempt: 39440 Artwork ID: 15155
== 404 or wrong page ==

IDs: 39440
Artw

== 404 or wrong page ==
Attempt: 39564 Artwork ID: 2507
== 404 or wrong page ==
Attempt: 39565 Artwork ID: 17397
== 404 or wrong page ==
Attempt: 39566 Artwork ID: 38049
== 404 or wrong page ==
Attempt: 39567 Artwork ID: 11702
== 404 or wrong page ==
Attempt: 39568 Artwork ID: 17118
== 404 or wrong page ==
Attempt: 39569 Artwork ID: 7559
== 404 or wrong page ==
Attempt: 39570 Artwork ID: 1808
== 404 or wrong page ==
Attempt: 39571 Artwork ID: 21702
== 404 or wrong page ==
Attempt: 39572 Artwork ID: 35141
== 404 or wrong page ==
Attempt: 39573 Artwork ID: 26193
== 404 or wrong page ==
Attempt: 39574 Artwork ID: 33384
== 404 or wrong page ==
Attempt: 39575 Artwork ID: 23012
== 404 or wrong page ==
Attempt: 39576 Artwork ID: 18292
== 404 or wrong page ==
Attempt: 39577 Artwork ID: 4549
== 404 or wrong page ==
Attempt: 39578 Artwork ID: 18258
== 404 or wrong page ==
Attempt: 39579 Artwork ID: 21512
== 404 or wrong page ==
Attempt: 39580 Artwork ID: 26113
== 404 or wrong page ==

IDs: 39580

== 404 or wrong page ==
Attempt: 39705 Artwork ID: 7856
== 404 or wrong page ==
Attempt: 39706 Artwork ID: 26461
== 404 or wrong page ==
Attempt: 39707 Artwork ID: 15414
== 404 or wrong page ==
Attempt: 39708 Artwork ID: 4453
== 404 or wrong page ==
Attempt: 39709 Artwork ID: 38996
== 404 or wrong page ==
Attempt: 39710 Artwork ID: 4490
== 404 or wrong page ==
Attempt: 39711 Artwork ID: 12178
== 404 or wrong page ==
Attempt: 39712 Artwork ID: 5250
== 404 or wrong page ==
Attempt: 39713 Artwork ID: 33281
== 404 or wrong page ==
Attempt: 39714 Artwork ID: 3727
Larry Rivers
Attempt: 39715 Artwork ID: 10317
== 404 or wrong page ==
Attempt: 39716 Artwork ID: 34807
== 404 or wrong page ==
Attempt: 39717 Artwork ID: 32954
== 404 or wrong page ==
Attempt: 39718 Artwork ID: 31246
== 404 or wrong page ==
Attempt: 39719 Artwork ID: 38284
== 404 or wrong page ==
Attempt: 39720 Artwork ID: 3286
== 404 or wrong page ==

IDs: 39720
Artworks: 1892
Images: 1892

Attempt: 39721 Artwork ID: 3746
== 404 o

== 404 or wrong page ==
Attempt: 39846 Artwork ID: 21243
== 404 or wrong page ==
Attempt: 39847 Artwork ID: 39252
== 404 or wrong page ==
Attempt: 39848 Artwork ID: 37
Julian Schnabel
Attempt: 39849 Artwork ID: 34757
== 404 or wrong page ==
Attempt: 39850 Artwork ID: 35348
== 404 or wrong page ==
Attempt: 39851 Artwork ID: 10587
== 404 or wrong page ==
Attempt: 39852 Artwork ID: 24504
== 404 or wrong page ==
Attempt: 39853 Artwork ID: 39884
== 404 or wrong page ==
Attempt: 39854 Artwork ID: 13035
== 404 or wrong page ==
Attempt: 39855 Artwork ID: 33270
== 404 or wrong page ==
Attempt: 39856 Artwork ID: 19558
== 404 or wrong page ==
Attempt: 39857 Artwork ID: 6973
== 404 or wrong page ==
Attempt: 39858 Artwork ID: 33271
== 404 or wrong page ==
Attempt: 39859 Artwork ID: 11708
== 404 or wrong page ==
Attempt: 39860 Artwork ID: 32806
== 404 or wrong page ==

IDs: 39860
Artworks: 1895
Images: 1895

Attempt: 39861 Artwork ID: 17261
== 404 or wrong page ==
Attempt: 39862 Artwork ID: 212
== 4

== 404 or wrong page ==
Attempt: 39986 Artwork ID: 6140
== 404 or wrong page ==
Attempt: 39987 Artwork ID: 1952
Vasily Kandinsky
Attempt: 39988 Artwork ID: 2082
Anselm Kiefer
Attempt: 39989 Artwork ID: 27651
== 404 or wrong page ==
Attempt: 39990 Artwork ID: 38698
== 404 or wrong page ==
Attempt: 39991 Artwork ID: 5697
== 404 or wrong page ==
Attempt: 39992 Artwork ID: 35741
== 404 or wrong page ==
Attempt: 39993 Artwork ID: 6717
== 404 or wrong page ==
Attempt: 39994 Artwork ID: 9144
== 404 or wrong page ==
Attempt: 39995 Artwork ID: 14628
== 404 or wrong page ==
Attempt: 39996 Artwork ID: 16049
== 404 or wrong page ==
Attempt: 39997 Artwork ID: 18024
== 404 or wrong page ==
Attempt: 39998 Artwork ID: 1639
== 404 or wrong page ==
Attempt: 39999 Artwork ID: 7296
== 404 or wrong page ==


In [11]:
# Check the final number of artworks.
len(gugg)

1901

In [12]:
# Take a look at a few artwork metadata entries.
gugg[:10]

[{'artwork_uid': 8303,
  'title': 'Composition No. 2',
  'date': '1946-01-01T05:00:00+00:00',
  'artist': 'Wallace Mitchell',
  'image_url': 'https://i0.wp.com/www.guggenheim.org/wp-content/uploads/1946/01/46.1048_ph_web-1.jpg'},
 {'artwork_uid': 493,
  'title': 'Dusk',
  'date': '1958-01-01T05:00:00+00:00',
  'artist': 'William Baziotes',
  'image_url': 'https://i2.wp.com/www.guggenheim.org/wp-content/uploads/1958/01/59.1544_ph_web-1.jpg'},
 {'artwork_uid': 18939,
  'title': 'Untitled (Dance Floor)',
  'date': '1996-01-01T05:00:00+00:00',
  'artist': 'Piotr Uklanski',
  'image_url': 'https://i2.wp.com/www.guggenheim.org/wp-content/uploads/1996/01/2006.72_ph_web-1.jpg'},
 {'artwork_uid': 1186,
  'title': 'Attirement of the Bride (La Toilette de la mariée)',
  'date': '1940-01-01T05:00:00+00:00',
  'artist': 'Max Ernst',
  'image_url': 'https://i2.wp.com/www.guggenheim.org/wp-content/uploads/1940/01/76.2553.78_ph_web-1.jpg'},
 {'artwork_uid': 13481,
  'title': 'untitled 2002 (he promise